### ztf_sim_introduction

This notebook illustrates basic use of the `ztf_sim` modules.

In [1]:
# hack to get the path right
import sys
sys.path.append('..')

In [2]:
import ztf_sim
from astropy.time import Time
import pandas as pd
import numpy as np
import astropy.units as u
import pylab as plt

/Users/ebellm/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


First we'll generate a test field grid.  You only need to do this the first time you run the simulator.

In [7]:
ztf_sim.fields.generate_test_field_grid()

Let's load the Fields object with the default field grid:

In [8]:
f = ztf_sim.fields.Fields()

The raw fieldid and coordinates are stored as a pandas Dataframe in the `.fields` attribute:

In [9]:
f.fields

,ra,dec,gridid
fieldid,,,
1,24.71429,-85.93846,0
2,76.14286,-85.93846,0
3,127.57143,-85.93846,0
4,179.00000,-85.93846,0
5,230.42857,-85.93846,0
6,281.85714,-85.93846,0
7,333.28571,-85.93846,0
8,12.84615,-79.01538,0
9,40.53846,-79.01538,0


Now let's calculate their altitude and azimuth at a specific time using the astropy.time.Time object:

In [10]:
f.alt_az(Time.now())

If you need enough precision such that this matters (~<10 arcsec), you can
download the latest IERS predictions by running:

    >>> from astropy.utils.data import download_file
    >>> from astropy.utils import iers
    >>> iers.IERS.iers_table = iers.IERS_A.open(download_file(iers.IERS_A_URL, cache=True))

 [astropy.coordinates.builtin_frames.utils]
If you need enough precision such that this matters (~<10 arcsec), you can
download the latest IERS predictions by running:

    >>> from astropy.utils.data import download_file
    >>> from astropy.utils import iers
    >>> iers.IERS.iers_table = iers.IERS_A.open(download_file(iers.IERS_A_URL, cache=True))


If you need enough precision such that this matters (~<10 arcsec), you can
download the latest IERS predictions by running:

    >>> from astropy.utils.data import download_file
    >>> from astropy.utils import iers
    >>> iers.IERS.iers_table = iers.IERS_A.open(download_file(iers.IERS_A_URL, cache=True))

 [astropy.coordinates.bui

,alt,az
fieldid,,
1,-32.010730,175.351458
2,-35.512555,175.804187
3,-37.343653,179.544015
4,-36.004918,183.606188
5,-32.600239,184.697566
6,-29.756256,182.268817
7,-29.501141,178.227523
8,-27.487707,169.105633
9,-32.352154,166.896886


Demonstrate accessing fields by `fieldid`:

In [15]:
w = f.fields['dec'] > -30.
f.fields[w].loc[853]

ra        141.50000
dec        66.36923
gridid      0.00000
Name: 853, dtype: float64

Calculating the overhead time (max of ha, dec, dome slews and readout time):

In [16]:
f.overhead_time(853,Time.now())

,overhead_time
fieldid,
1,133.923075
2,133.923075
3,133.923075
4,133.923075
5,133.923075
6,133.923075
7,133.923075
8,128.153842
9,128.153842


In [ ]:
f = ztf_sim.fields.Fields()
Exposure_time = 60*u.second
Night_length=9*u.h


time0 = Time('2015-09-10 20:00:00') + 7*u.h
time = time0
f.fields = f.fields.join(pd.DataFrame(np.zeros(len(f.fields)),columns=['observed']))
f.fields = f.fields.join(pd.DataFrame(np.zeros(len(f.fields)),columns=['possibleToObserve']))

def observe(f, nightStart):
    time=nightStart
    goodAltitude = f.alt_az(time)['alt'] > 20
    shouldObserve = f.fields['observed'] == 0
    good = goodAltitude & shouldObserve #& f.alt_az(time+1*u.h)['alt'] < 20 # start with a field which won't be observable later
    
    if np.all(good) is False:
        good = goodAltitude & shouldObserve
        
    fid = f.fields[good].iloc[0].name
    f.fields['observed'][fid]+=1
    f.fields['possibleToObserve'][goodAltitude] = 1
    time += Exposure_time

    while time < nightStart + Night_length:
        goodAltitude = f.alt_az(time)['alt'] > 20
        shouldObserve = f.fields['observed'] == 0
        good = goodAltitude & shouldObserve
        f.fields['possibleToObserve'][goodAltitude] = 1
        
        if not np.any(good):
            time += 60*u.s
            continue
            
        slewTime = f.overhead_time(fid,time)[good]
        fid = int(slewTime.idxmin())
        # print slewTime['overhead_time'][fid]
        time += Exposure_time + slewTime['overhead_time'][fid]*u.second
        f.fields['observed'][fid]+=1
        # print time-7*u.h
        

# First night
observe(f,time)
fieldsPossible = np.sum(f.fields['possibleToObserve'])
print fieldsPossible
fieldsObserved = np.sum(f.fields['observed'])
print fieldsObserved
meanTime = (Night_length.to(u.s)-fieldsObserved*Exposure_time)/(fieldsObserved-1)
print meanTime

# Second night
time=time0+24*u.h
observe(f,time)
fieldsPossible = np.sum(f.fields['possibleToObserve'])
print fieldsPossible
fieldsObserved = np.sum(f.fields['observed'])
print fieldsObserved
meanTime = (2*Night_length.to(u.s)-fieldsObserved*Exposure_time)/(fieldsObserved-1)
print meanTime

In [ ]:
for dec in np.append(np.linspace(-90,90,10),0):
    ra=np.linspace(0, 360,1000)
    x,y = raDec2xy(ra,dec)
    plt.plot(x,y,'k')
    
for ra in np.linspace(0,360,10):
    dec=np.linspace(-90, 90,1000)
    x,y = raDec2xy(ra,dec)
    plt.plot(x,y,'k')
    
x,y = raDec2xy(f.fields['ra'],f.fields['dec'])
plt.plot(x,y,'o',color=(.8,.8,.8))    
plt.show()

In [6]:
def raDec2xy(ra,dec):
    # Using Aitoff projections (from Wiki) returns x-y coordinates on a plane of RA and Dec
    theta = np.deg2rad(dec)
    phi = np.deg2rad(ra)-np.pi #the range is [-pi,pi]
    alpha=np.arccos(np.cos(theta)*np.cos(phi/2))
    x=2*np.cos(theta)*np.sin(phi/2)/np.sinc(alpha/np.pi) # The python's sinc is normalazid, hence the /pi
    y=np.sin(theta)/np.sinc(alpha/np.pi)
    return x,y